# Tải ảnh và cài đặt một số thư viện


In [ ]:
# get image
%cd /content
!mkdir test_images
%cd test_images
!wget https://aambassador.com/images/car/car-3z.jpg


In [ ]:
!pip install tensorflow tf2onnx

In [ ]:
!pip install onnxruntime

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import onnxruntime

img_path = '/content/test_images/car-3z.jpg'

img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
print(x.shape)
y = x[0]
print(y.shape)

(1, 224, 224, 3)
(224, 224, 3)


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x[0])
plt.show()

# Convert model Resnet50 từ tensoflow sang ONNX

In [ ]:
model = ResNet50(weights='imagenet')

preds = model.predict(x)
print('Keras Predicted:', decode_predictions(preds, top=3)[0])
model.save(os.path.join("/tmp", model.name))

In [ ]:
#convert
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = model.name + ".onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

# Kiểm tra model

In [ ]:
#Run the ONNX model
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": x})

print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])

# make sure ONNX and keras have the same results
np.testing.assert_allclose(preds, onnx_pred[0], rtol=1e-5)

ONNX Predicted: [('n03930630', 'pickup', 0.29420894), ('n02814533', 'beach_wagon', 0.21882463), ('n03459775', 'grille', 0.19729643)]
